In [2]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Dropout
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt

In [3]:
(Xtrain,ytrain),(Xtest,ytest) = mnist.load_data()
Xtrain = Xtrain/255
Xtest = Xtest/255
print(Xtrain.shape)

11490434/11490434 [==============================] - 1s 0us/step
(60000, 28, 28)


In [15]:
#4d input required:
Xtrain = Xtrain.reshape(Xtrain.shape[0],28,28,1)
Xtest = Xtest.reshape(Xtest.shape[0],28,28,1)
print(Xtest.shape)

(10000, 28, 28, 1)


In [6]:
dataGen = ImageDataGenerator(
    rotation_range = 10,
    width_shift_range = 0.1, #Up to 10% of total width randomly changes
    height_shift_range = 0.1,
    zoom_range = 0.1
)

dataGen.fit(Xtrain)

In [7]:
initialLR = 0.01
batchSize = 64
epochs = 20
def scheduleLR(epoch,LR):
    if epoch > 0 and epoch%3==0:
        return LR/2
    return LR

In [13]:
DLmodel = Sequential([
    Flatten(input_shape=(28,28,1)),
    Dense(500,activation="relu"),
    Dropout(0.2),
    Dense(250,activation="relu"),
    Dropout(0.2),
    Dense(10,activation="softmax")
])
optimiser = SGD(learning_rate=initialLR)
DLmodel.compile(optimizer=optimiser,loss="categorical_crossentropy",metrics=["accuracy"])

In [9]:
earlyStop = EarlyStopping(
    monitor="val_loss",#checks loss of test data to see when it stops improving
    patience = 3,#3 chances(epochs) to improve loss
    restore_best_weights = True,#After stopping goes back to best version
)
LRScheduler = LearningRateScheduler(scheduleLR)

In [14]:
fittedModel = DLmodel.fit(dataGen.flow(Xtrain,ytrain,batch_size=batchSize),
            epochs=epochs,validation_data=(Xtest,ytest),
            callbacks=[earlyStop,LRScheduler])

Epoch 1/20


ValueError: in user code:

    File "c:\Users\morga\anaconda3\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\morga\anaconda3\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\morga\anaconda3\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\morga\anaconda3\lib\site-packages\keras\engine\training.py", line 994, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\Users\morga\anaconda3\lib\site-packages\keras\engine\training.py", line 1052, in compute_loss
        return self.compiled_loss(
    File "c:\Users\morga\anaconda3\lib\site-packages\keras\engine\compile_utils.py", line 265, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "c